# Align and remove under/overexposed images

This notebook removes any under/overexposed frames from timelapse experiments and aligns the images. 

### The aligned images will be used later in the object localisation and tracking steps. 

The structure of this notebook is:

1. Load images using the octopuslite dask loader.
2. Find over/underexposed images by measuring each channel and frame for average pixel intensity.
3. Select a reference channel to center the alignment on
4. Register alignment and save out transformation tensor
5. (Optional) Apply transformation matrix to all channels and save out images
6. Check images using Napari
7. Function to iterate over many experiments, many positions

In [13]:
import os
import glob
import enum
import numpy as np
from pystackreg import StackReg
from skimage.io import imsave, imread
from tqdm.auto import tqdm
from octopuslite import DaskOctopusLiteLoader
from skimage import transform as tf

In [2]:
def image_generator(files, crop = None):
    """
    Generator function for iterative procesessing of image files
    """
    
    if crop is None:
        for filename in files:
            img = imread(filename)
            yield img
    else:
        for filename in files:
            img = imread(filename)
            img = crop_image(img, crop)
            yield img

## 1. Find images, organise and load using octopuslite

Define root directory and specific experiment and location to align

In [3]:
root_dir = '/home/nathan/data/kraken/ras'
expt = 'ND0012'
pos = 'Pos0'

Create new subdir for image files and move them all there. This is so that the miscelleanous non-image files (such as transformation matrices and tracking files) are easy to access later on and not lost amongst many single frame timelapse images.

In [4]:
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
if not os.path.exists(image_path):
    os.mkdir(image_path)
    files = sorted(glob.glob(f'{root_dir}/{expt}/{pos}/*.tif'))
    for file in files:
        os.rename(file, file.replace(f'{pos}', f'{pos}/{pos}_images'))

Lazily load image array and associated information using dask octopuslite-loader and display channels found. Note the optional background removal is not invoked at this stage.

In [5]:
images = DaskOctopusLiteLoader(image_path, remove_background = False)
print([channel.name for channel in images.channels])

['BRIGHTFIELD', 'GFP', 'RFP', 'IRFP', 'MASK']


## 2. Identify under/overexposed images and display average channel brightness

In [ ]:
%%time
# pixel range criteria
max_pixel, min_pixel = 200, 2
# set empty dict arrays for mean values 
mean_arrays = {}
# set for dodgy frames (only unique entries)
dodgy_frame_list = set([])
#iterate over channels
for channel in tqdm(images.channels, desc = f'Finding mean values of image channels'):
    # find mean value of each frame in each channel
    mean_arrays[channel.name] = [np.mean(img) for img in image_generator(images.files(channel.name))]
    # iterate over frames
    for frame, mean_value in enumerate(mean_arrays[channel.name]):
        # check to see if mean frame pixel value meets criteria
        if max_pixel < mean_value or mean_value < min_pixel:
            # if so add to delete list
            dodgy_frame_list.add(frame)
# format delete list to only include single values
dodgy_frame_list = list(dodgy_frame_list)
print('Number of under/over-exposed frames:', len(dodgy_frame_list))

Finding mean values of image channels:   0%|          | 0/5 [00:00<?, ?it/s]

### 2a. Filter blanks from main image folder into separate directory

This step is optional as there is a parameter within `DaskOctopusLiteLoader` that filters the images, but employing that every time you load images is time consuming for large data sets

In [30]:
# check if blanks dir exists and make if not
if not os.path.exists(f'{root_dir}/{expt}/{pos}/{pos}_blanks'):
    os.mkdir(f'{root_dir}/{expt}/{pos}/{pos}_blanks')
# move blank images into this directory
for channel in images.channels:
    for f in images.files(channel.name):
        for i in dodgy_frame_list:
            if str(i).zfill(9) in f:
                os.rename(f, f.replace('_images', '_blanks'))
# reload image arrays now that blanks filtered
images = DaskOctopusLiteLoader(image_path, remove_background = False)
images['gfp']

dask.array<stack, shape=(1638, 1352, 1688), dtype=uint8, chunksize=(1, 1352, 1688), chunktype=numpy.ndarray>

## 3. Select reference image to base alignment around

Display the average intensities of each channel. The automatically-measured brightest channel isn't necessarily the best.

In [31]:
print('Average channel brightness for selection of reference image:')
for channel in images.channels:
    print(f'{channel.value}: {channel.name}:', np.mean(mean_arrays[channel.name]))

Average channel brightness for selection of reference image:
0: BRIGHTFIELD: 40.96403863128827
1: GFP: 45.97970142886046
2: RFP: 6.969645431239206
3: IRFP: 47.732877102157886


In [33]:
# manually select reference channel by adding index
reference_channel = images.channels[1]
# automatically select reference channel from max average pixel value (ie. brightest channel)
#reference_channel = images.channels[max([(channel.value, np.mean(mean_arrays[channel.name])) for channel in images.channels])[0]]
reference_channel.name

'GFP'

#### 3a. Set cropped area of reference image to base alignment around 
Cropping as alignment struggles on large arrays such as `shape = (1200,1353,1682)`, this step is optional but you will still need to run `.compute()` on the dask array to load the image into memory to perform the alignment.

In [37]:
%%time
# crop central window out of reference image
reference_image = DaskOctopusLiteLoader(image_path, 
                                        crop = (500, 500)
                                       )[reference_channel.name].compute()
reference_image.shape

Using cropping: (500, 500)
CPU times: user 4min 53s, sys: 3min 39s, total: 8min 32s
Wall time: 55.6 s


(1638, 500, 500)

## 4. Register alignment and save out transformation tensor
Transformation tensor is a 3D series of transformation matrices over time

In [42]:
%%time
# create operator using transformation type (translation)
sr = StackReg(StackReg.TRANSLATION) 

# register each frame to the previous as transformation matrices/tensor
transform_tensor = sr.register_stack(reference_image, reference = 'previous')

# save out transform tensor
np.save(f'{root_dir}/{expt}/{pos}/{reference_channel.name.lower()}_transform_tensor.npy', transform_tensor)

/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/pystackreg/pystackreg.py:379: UserWarning: Detected axis 2 as the possible time axis for the stack due to its low variability, but axis 0 was supplied for registration. Are you sure you supplied the correct axis?
  warnings.warn(


CPU times: user 3min 19s, sys: 3.78 s, total: 3min 23s
Wall time: 3min 23s


In [47]:
transform_tensor.shape

(1638, 3, 3)

## 5. (Optional) Apply transformation matrix to all channels and save out images in separate directory

Consumes a lot of time and space to replicate images with minor translational shifts, it is advised to just use the transform parameter in the `DaskOctopusLiteLoader`. 

In [14]:
%%time
### iterating over channels
# create aligned image dir if does not exist 
if not os.path.exists(f'{root_dir}/{expt}/{pos}/{pos}_aligned'):
    os.mkdir(f'{root_dir}/{expt}/{pos}/{pos}_aligned')
# iterate over channels
for channel in images.channels:
    #iterate over all images in channel
    for i in tqdm(range(len(transform_tensor)), 
                  desc = f'Aligning {channel.name.lower()} channel {channel.value+1}/{len(images.channels)}'):
        # load specific transform matrix for that frame
        transform_matrix = tf.EuclideanTransform(matrix = transform_tensor[i,...],
                                                 rotation = None)
        # transform image
        transformed_image = (tf.warp(images[channel.name][i,...].compute(), 
                                     transform_matrix, preserve_range=True)).astype(np.uint8)
        # set transformed image pathname by editing base dir
        fn = images.files(channel.name)[i].replace('_images', '_aligned')
        # save trans image out
        imsave(fn, transformed_image, check_contrast=False)

Aligning irfp channel 4/4: 100%|██████████| 1067/1067 [03:04<00:00,  5.79it/s]

CPU times: user 5min 24s, sys: 51.4 s, total: 6min 15s
Wall time: 11min 45s


## 6. Check alignment using Napari

In [53]:
import napari

In [56]:
aligned_images = DaskOctopusLiteLoader(image_path, 
                                       #crop = (1200,1600), 
                                       transforms = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy',
                                       remove_background=False)
viewer = napari.Viewer()
for channel in aligned_images.channels:
    viewer.add_image(aligned_images[channel.name], 
                     name = channel.name, 
                     blending = 'additive', 
                     contrast_limits = [0,255])

## Batch execute

Do all of the above but for many experiment IDs and many positions

In [ ]:
root_dir = '/home/nathan/data/kraken/ras'
expt_list = ['ND0010']
max_pixel = 200
min_pixel = 2
alignment_channel = 'gfp'
crop_area = 500
save_out_images = False ### this does not save out a copy of the images, only the transformation matrix
overwrite = False ### this checks for any prexisting transformations and does not overwrite

# trying reference to first image first

In [2]:
from multiprocessing import Pool
cpus = os.cpu_count()
cpus

12

In [3]:
from natsort import natsorted

In [4]:
expt = 'ND0010'
root_dir = '/home/nathan/data/kraken/ras'
pos_list = natsorted([pos for pos in os.listdir(f'{root_dir}/{expt}') 
               if 'Pos' in pos 
               and os.path.isdir(f'{root_dir}/{expt}/{pos}')]  )


In [5]:
def align(pos):
    
    expt = 'ND0010'
    root_dir = '/home/nathan/data/kraken/ras'
    
    print(f'Starting {expt}/{pos}')

    ### create new subdir of for raw files and move them all there
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    
    ### pre load files from raw file dir 
    images = DaskOctopusLiteLoader(image_path, remove_background= False)
    
    crop_area = 500
    # crop central window out of reference image with blanks removed
    reference_image = DaskOctopusLiteLoader(image_path, 
                                            crop = (crop_area, crop_area), 
                                            transforms=None
                                           )['gfp'].compute() 
    ## check that image is sequentially correct
    
    ### Register alignment
    print('Registering alignment for', pos, expt)
    # create operator using transformation type (translation)
    sr = StackReg(StackReg.TRANSLATION) 
    # register each frame to the previous as transformation matrices/tensor
    transform_tensor = sr.register_stack(reference_image, reference = 'first')
    # save out transform tensor
    np.save(f'{root_dir}/{expt}/{pos}/gfp_transform_tensor_2.npy', transform_tensor)
    print(f'/{expt}/{pos}/ saved')
    return


In [31]:
def show_progress(current_iteration, end_iteration):
    print(f"Registering {current_iteration} of {end_iteration} images")

def align_overnight(pos):
    expt = 'ND0010'
    root_dir = '/home/nathan/data/kraken/ras'
    if os.path.exists(f'{root_dir}/{expt}/{pos}/gfp_transform_tensor_8.npy'):
        print('already aligned, skipping', expt, pos)
        return

    
    print(f'Starting {expt}/{pos}')

    ### create new subdir of for raw files and move them all there
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    
    ### pre load files from raw file dir 
    images = DaskOctopusLiteLoader(image_path, remove_background= False)
    
    #crop_area = 500
    
    N_files = len(images['gfp'])
    transform_tensor_sum = []
    for chunk in tqdm(range(0,N_files,int(N_files/10))):
    # crop central window out of reference image with blanks removed
        reference_image = DaskOctopusLiteLoader(image_path, 
                                                #crop = (crop_area, crop_area), 
                                              # transforms='', 
                                               )['gfp'][chunk:chunk+int(N_files/10)].compute() 
        ### Register alignment
        print('Registering alignment for', pos, expt)
        # create operator using transformation type (translation)# if self._transformer is not None
        sr = StackReg(StackReg.TRANSLATION) 
        # register each frame to the previous as transformation matrices/tensor
        transform_tensor = sr.register_stack(reference_image, reference = 'previous', progress_callback=show_progress)
        transform_tensor_sum.append(transform_tensor)
    #concatenate different chunks of trans tensor
    transform_tensor = np.concatenate(transform_tensor_sum, axis=0)
    #reshape
    #transform_tensor = np.reshape(transform_tensor, (len(transform_tensor)*len(transform_tensor[0]), 3, 3))
    # save out transform tensor
    print('shape:', transform_tensor.shape)
    np.save(f'{root_dir}/{expt}/{pos}/gfp_transform_tensor_8.npy', transform_tensor)
    print(f'/{expt}/{pos}/ saved')
    return

In [34]:
for pos in pos_list:
    expt = 'ND0010'
    root_dir = '/home/nathan/data/kraken/ras'
    if os.path.exists(f'{root_dir}/{expt}/{pos}/gfp_transform_tensor_8.npy'):
        print('already aligned, skipping', expt, pos)
        continue

    
    print(f'Starting {expt}/{pos}')

    ### create new subdir of for raw files and move them all there
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    
    ### pre load files from raw file dir 
    images = DaskOctopusLiteLoader(image_path, remove_background= False)
    
    #crop_area = 500
    
    N_files = len(images['gfp'])
    transform_tensor_sum = []
    for chunk in tqdm(range(0,N_files,int(N_files/10))):
    # crop central window out of reference image with blanks removed
        reference_image = DaskOctopusLiteLoader(image_path, 
                                                #crop = (crop_area, crop_area), 
                                              # transforms='', 
                                               )['gfp'][chunk:chunk+int(N_files/10)].compute() 
        ### Register alignment
        print('Registering alignment for', pos, chunk,chunk+int(N_files/10), 'frames')
        # create operator using transformation type (translation)# if self._transformer is not None
        sr = StackReg(StackReg.TRANSLATION) 
        # register each frame to the previous as transformation matrices/tensor
        transform_tensor = sr.register_stack(reference_image, reference = 'previous', progress_callback=show_progress)
        transform_tensor_sum.append(transform_tensor)
    #concatenate different chunks of trans tensor
    transform_tensor = np.concatenate(transform_tensor_sum, axis=0)
    #reshape
    #transform_tensor = np.reshape(transform_tensor, (len(transform_tensor)*len(transform_tensor[0]), 3, 3))
    # save out transform tensor
    print('shape:', transform_tensor.shape)
    np.save(f'{root_dir}/{expt}/{pos}/gfp_transform_tensor_8.npy', transform_tensor)
    print(f'/{expt}/{pos}/ saved')

already aligned, skipping ND0010 Pos0
already aligned, skipping ND0010 Pos1
Starting ND0010/Pos2


  0%|          | 0/11 [00:00<?, ?it/s]

/home/nathan/analysis/octopuslite-reader/octopuslite/reader.py:147: UserWarning: Background removal works best on cropped, aligned image. Will fail on uncropped, aligned images due to border effect.
  warnings.warn(


Registering alignment for Pos2 0 118 frames


/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/pystackreg/pystackreg.py:379: UserWarning: Detected axis 1 as the possible time axis for the stack due to its low variability, but axis 0 was supplied for registration. Are you sure you supplied the correct axis?
  warnings.warn(


...

Registering alignment for Pos11 0 119 frames
Registering 1 of 118 images
Registering 2 of 118 images
Registering 3 of 118 images
Registering 4 of 118 images
Registering 5 of 118 images
Registering 6 of 118 images
Registering 7 of 118 images
Registering 8 of 118 images
Registering 9 of 118 images
Registering 10 of 118 images
Registering 11 of 118 images
Registering 12 of 118 images
Registering 13 of 118 images
Registering 14 of 118 images
Registering 15 of 118 images
Registering 16 of 118 images
Registering 17 of 118 images
Registering 18 of 118 images
Registering 19 of 118 images
Registering 20 of 118 images
Registering 21 of 118 images
Registering 22 of 118 images
Registering 23 of 118 images
Registering 24 of 118 images
Registering 25 of 118 images
Registering 26 of 118 images
Registering 27 of 118 images
Registering 28 of 118 images
Registering 29 of 118 images
Registering 30 of 118 images
Registering 31 of 118 images
Registering 32 of 118 images
Registering 33 of 118 images
Registe

Registering 43 of 118 images
Registering 44 of 118 images
Registering 45 of 118 images
Registering 46 of 118 images
Registering 47 of 118 images
Registering 48 of 118 images
Registering 49 of 118 images
Registering 50 of 118 images
Registering 51 of 118 images
Registering 52 of 118 images
Registering 53 of 118 images
Registering 54 of 118 images
Registering 55 of 118 images
Registering 56 of 118 images
Registering 57 of 118 images
Registering 58 of 118 images
Registering 59 of 118 images
Registering 60 of 118 images
Registering 61 of 118 images
Registering 62 of 118 images
Registering 63 of 118 images
Registering 64 of 118 images
Registering 65 of 118 images
Registering 66 of 118 images
Registering 67 of 118 images
Registering 68 of 118 images
Registering 69 of 118 images
Registering 70 of 118 images
Registering 71 of 118 images
Registering 72 of 118 images
Registering 73 of 118 images
Registering 74 of 118 images
Registering 75 of 118 images
Registering 76 of 118 images
Registering 77

Registering 86 of 118 images
Registering 87 of 118 images
Registering 88 of 118 images
Registering 89 of 118 images
Registering 90 of 118 images
Registering 91 of 118 images
Registering 92 of 118 images
Registering 93 of 118 images
Registering 94 of 118 images
Registering 95 of 118 images
Registering 96 of 118 images
Registering 97 of 118 images
Registering 98 of 118 images
Registering 99 of 118 images
Registering 100 of 118 images
Registering 101 of 118 images
Registering 102 of 118 images
Registering 103 of 118 images
Registering 104 of 118 images
Registering 105 of 118 images
Registering 106 of 118 images
Registering 107 of 118 images
Registering 108 of 118 images
Registering 109 of 118 images
Registering 110 of 118 images
Registering 111 of 118 images
Registering 112 of 118 images
Registering 113 of 118 images
Registering 114 of 118 images
Registering 115 of 118 images
Registering 116 of 118 images
Registering 117 of 118 images
Registering 118 of 118 images
Registering alignment fo

Registering 9 of 118 images
Registering 10 of 118 images
Registering 11 of 118 images
Registering 12 of 118 images
Registering 13 of 118 images
Registering 14 of 118 images
Registering 15 of 118 images
Registering 16 of 118 images
Registering 17 of 118 images
Registering 18 of 118 images
Registering 19 of 118 images
Registering 20 of 118 images
Registering 21 of 118 images
Registering 22 of 118 images
Registering 23 of 118 images
Registering 24 of 118 images
Registering 25 of 118 images
Registering 26 of 118 images
Registering 27 of 118 images
Registering 28 of 118 images
Registering 29 of 118 images
Registering 30 of 118 images
Registering 31 of 118 images
Registering 32 of 118 images
Registering 33 of 118 images
Registering 34 of 118 images
Registering 35 of 118 images
Registering 36 of 118 images
Registering 37 of 118 images
Registering 38 of 118 images
Registering 39 of 118 images
Registering 40 of 118 images
Registering 41 of 118 images
Registering 42 of 118 images
Registering 43 

Registering 52 of 118 images
Registering 53 of 118 images
Registering 54 of 118 images
Registering 55 of 118 images
Registering 56 of 118 images
Registering 57 of 118 images
Registering 58 of 118 images
Registering 59 of 118 images
Registering 60 of 118 images
Registering 61 of 118 images
Registering 62 of 118 images
Registering 63 of 118 images
Registering 64 of 118 images
Registering 65 of 118 images
Registering 66 of 118 images
Registering 67 of 118 images
Registering 68 of 118 images
Registering 69 of 118 images
Registering 70 of 118 images
Registering 71 of 118 images
Registering 72 of 118 images
Registering 73 of 118 images
Registering 74 of 118 images
Registering 75 of 118 images
Registering 76 of 118 images
Registering 77 of 118 images
Registering 78 of 118 images
Registering 79 of 118 images
Registering 80 of 118 images
Registering 81 of 118 images
Registering 82 of 118 images
Registering 83 of 118 images
Registering 84 of 118 images
Registering 85 of 118 images
Registering 86

In [39]:
for chunk in tqdm(range(0,N_files,int(N_files/10))):
    print(chunk,chunk+int(N_files/10))

  0%|          | 0/11 [00:00<?, ?it/s]

0 119
119 238
238 357
357 476
476 595
595 714
714 833
833 952
952 1071
1071 1190
1190 1309


In [41]:
for chunk in tqdm(range(1,N_files,int(N_files/10))):
    print(chunk-1,chunk+int(N_files/10))

  0%|          | 0/10 [00:00<?, ?it/s]

0 120
119 239
238 358
357 477
476 596
595 715
714 834
833 953
952 1072
1071 1191


In [ ]:
for pos in pos_list:
    expt = 'ND0010'
    root_dir = '/home/nathan/data/kraken/ras'
    
    print(f'Starting {expt}/{pos}')

    ### create new subdir of for raw files and move them all there
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    
    ### pre load files from raw file dir 
    images = DaskOctopusLiteLoader(image_path, remove_background= False)
    
    #crop_area = 500
    
    N_files = len(images['gfp'])
    transform_tensor_sum = []
    for chunk in tqdm(range(1,N_files,int(N_files/10))):
    # crop central window out of reference image with blanks removed
        reference_image = DaskOctopusLiteLoader(image_path, 
                                                #crop = (crop_area, crop_area), 
                                                remove_background=False, 
                                               )['gfp'][chunk-1:chunk+int(N_files/10)].compute() 
        ### Register alignment
        print('Registering alignment for', pos, chunk,chunk+int(N_files/10), 'frames')
        # create operator using transformation type (translation)# if self._transformer is not None
        sr = StackReg(StackReg.TRANSLATION) 
        # register each frame to the previous as transformation matrices/tensor
        transform_tensor = sr.register_stack(reference_image, reference = 'previous', progress_callback=show_progress)
        transform_tensor_sum.append(transform_tensor)
    #concatenate different chunks of trans tensor
    transform_tensor = np.concatenate(transform_tensor_sum, axis=0)
    #reshape
    #transform_tensor = np.reshape(transform_tensor, (len(transform_tensor)*len(transform_tensor[0]), 3, 3))
    # save out transform tensor
    print('shape:', transform_tensor.shape)
    np.save(f'{root_dir}/{expt}/{pos}/gfp_transform_tensor_14.npy', transform_tensor)
    print(f'/{expt}/{pos}/ saved')

Starting ND0010/Pos0


  0%|          | 0/10 [00:00<?, ?it/s]

Registering alignment for Pos0 1 120 frames


/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/pystackreg/pystackreg.py:379: UserWarning: Detected axis 1 as the possible time axis for the stack due to its low variability, but axis 0 was supplied for registration. Are you sure you supplied the correct axis?
  warnings.warn(


Registering 1 of 119 images
Registering 2 of 119 images
Registering 3 of 119 images
Registering 4 of 119 images
Registering 5 of 119 images
Registering 6 of 119 images
Registering 7 of 119 images
Registering 8 of 119 images
Registering 9 of 119 images
Registering 10 of 119 images
Registering 11 of 119 images
Registering 12 of 119 images
Registering 13 of 119 images
Registering 14 of 119 images
Registering 15 of 119 images
Registering 16 of 119 images
Registering 17 of 119 images
Registering 18 of 119 images
Registering 19 of 119 images
Registering 20 of 119 images
Registering 21 of 119 images
Registering 22 of 119 images
Registering 23 of 119 images
Registering 24 of 119 images
Registering 25 of 119 images
Registering 26 of 119 images
Registering 27 of 119 images
Registering 28 of 119 images
Registering 29 of 119 images
Registering 30 of 119 images
Registering 31 of 119 images
Registering 32 of 119 images
Registering 33 of 119 images
Registering 34 of 119 images
Registering 35 of 119 i

...

  0%|          | 0/11 [00:00<?, ?it/s]

Registering alignment for Pos1 1 120 frames
Registering 1 of 119 images
Registering 2 of 119 images
Registering 3 of 119 images
Registering 4 of 119 images
Registering 5 of 119 images
Registering 6 of 119 images
Registering 7 of 119 images
Registering 8 of 119 images
Registering 9 of 119 images
Registering 10 of 119 images
Registering 11 of 119 images
Registering 12 of 119 images
Registering 13 of 119 images
Registering 14 of 119 images
Registering 15 of 119 images
Registering 16 of 119 images
Registering 17 of 119 images
Registering 18 of 119 images
Registering 19 of 119 images
Registering 20 of 119 images
Registering 21 of 119 images
Registering 22 of 119 images
Registering 23 of 119 images
Registering 24 of 119 images
Registering 25 of 119 images
Registering 26 of 119 images
Registering 27 of 119 images
Registering 28 of 119 images
Registering 29 of 119 images
Registering 30 of 119 images
Registering 31 of 119 images
Registering 32 of 119 images
Registering 33 of 119 images
Register

Registering 41 of 119 images
Registering 42 of 119 images
Registering 43 of 119 images
Registering 44 of 119 images
Registering 45 of 119 images
Registering 46 of 119 images
Registering 47 of 119 images
Registering 48 of 119 images
Registering 49 of 119 images
Registering 50 of 119 images
Registering 51 of 119 images
Registering 52 of 119 images
Registering 53 of 119 images
Registering 54 of 119 images
Registering 55 of 119 images
Registering 56 of 119 images
Registering 57 of 119 images
Registering 58 of 119 images
Registering 59 of 119 images
Registering 60 of 119 images
Registering 61 of 119 images
Registering 62 of 119 images
Registering 63 of 119 images
Registering 64 of 119 images
Registering 65 of 119 images
Registering 66 of 119 images
Registering 67 of 119 images
Registering 68 of 119 images
Registering 69 of 119 images
Registering 70 of 119 images
Registering 71 of 119 images
Registering 72 of 119 images
Registering 73 of 119 images
Registering 74 of 119 images
Registering 75

Registering 82 of 119 images
Registering 83 of 119 images
Registering 84 of 119 images
Registering 85 of 119 images
Registering 86 of 119 images
Registering 87 of 119 images
Registering 88 of 119 images
Registering 89 of 119 images
Registering 90 of 119 images
Registering 91 of 119 images
Registering 92 of 119 images
Registering 93 of 119 images
Registering 94 of 119 images
Registering 95 of 119 images
Registering 96 of 119 images
Registering 97 of 119 images
Registering 98 of 119 images
Registering 99 of 119 images
Registering 100 of 119 images
Registering 101 of 119 images
Registering 102 of 119 images
Registering 103 of 119 images
Registering 104 of 119 images
Registering 105 of 119 images
Registering 106 of 119 images
Registering 107 of 119 images
Registering 108 of 119 images
Registering 109 of 119 images
Registering 110 of 119 images
Registering 111 of 119 images
Registering 112 of 119 images
Registering 113 of 119 images
Registering 114 of 119 images
Registering 115 of 119 image

Registering 2 of 119 images
Registering 3 of 119 images
Registering 4 of 119 images
Registering 5 of 119 images
Registering 6 of 119 images
Registering 7 of 119 images
Registering 8 of 119 images
Registering 9 of 119 images
Registering 10 of 119 images
Registering 11 of 119 images
Registering 12 of 119 images
Registering 13 of 119 images
Registering 14 of 119 images
Registering 15 of 119 images
Registering 16 of 119 images
Registering 17 of 119 images
Registering 18 of 119 images
Registering 19 of 119 images
Registering 20 of 119 images
Registering 21 of 119 images
Registering 22 of 119 images
Registering 23 of 119 images
Registering 24 of 119 images
Registering 25 of 119 images
Registering 26 of 119 images
Registering 27 of 119 images
Registering 28 of 119 images
Registering 29 of 119 images
Registering 30 of 119 images
Registering 31 of 119 images
Registering 32 of 119 images
Registering 33 of 119 images
Registering 34 of 119 images
Registering 35 of 119 images
Registering 36 of 119 